In [ ]:
# default_exp function_approximation

# Function Approximation

> function approximation playground

`Go to Runtime -> Change runtime type and make sure Hardward accelerator is set to GPU`

In [ ]:
# export
from abc import ABC, abstractmethod
from fastcore.test import *
import jsmltools
from jsmltools import data_generator
from nbdev.showdoc import *
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

test_eq(tf.__version__, "2.2.0")

In [ ]:
# export

class KerasPipeline(ABC):
    '''Scafolding for a model building pipeline.
    
    This base class will contain scafolding for all pipelines
    used. The goal of this base class is mainly to add safeguard
    and assumption enforcement.
    '''
    def __init__(self):
        # safe guard: to ensure that various session don't collide
        keras.backend.clear_session()
        self.train_x, self.train_y, self.test_x, self.test_y = None, None, None, None
        self.model = None
        self.evaluation = None
        self.train_history = None
    
    @abstractmethod
    def get_data(self):
        pass

    @abstractmethod
    def build_model(self):
        pass
    
    @abstractmethod
    def run_training(self):
        pass
    
    @abstractmethod
    def evaluate_pipeline(self):
        pass
    
    def keras_model_summary(self):
        assert self.model is not None
        return self.model.summary()
    
    @abstractmethod    
    def __call__(self):
        # checking that self.get_data() is implimented as expected
        assert list(map(lambda x: x is not None, [self.train_x, self.train_y, self.test_x, self.test_y]))  == [True] * 4 

        # checking that self.model() is implimented as expected
        assert self.model is not None

        # checking that self.train() is implimented as expected
        assert self.train_history is not None
        assert self.model._is_compiled == True

        # checking that self.evaluate() is implimented as expected
        assert self.evaluation is not None

        # TODO (jovsa): run tests that ensure pipeline trains model above a benchmark
        

### Using High Level APIs

#### Using tf.keras Functional API

In [ ]:
#export
class CustomLoss(keras.losses.Loss):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
        
        def call(self, y_true, y_pred):
            pass
        
        def get_config(self):
            pass

class CustomMetrics(keras.metrics.Metric):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def update_state(self, y_true, y_pred, sample_weight=None):
        pass
    
    def result(self):
        pass
    
    def get_config(self):
        pass


class HighLevelPipeline(KerasPipeline):
    def __init__(self):
        super().__init__()
    
    def get_data(self):
        additional_y, train_n = 0, 1000
        data = data_generator.MNISTDataGenerator(additional_y)
        datasets  = data(train_n)
        self.train_x, self.train_y, self.test_x, self.test_y = datasets

    def build_model(self):
        inputs = keras.Input(shape=(784, 1))
        # Rescale images to [0, 1]
        x = Rescaling(scale=1./255)(inputs)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(units=128, activation='relu')(x)
        x = keras.layers.Dense(units=128, activation='relu')(x)
        outputs = keras.layers.Dense(units=10, activation='softmax')(x)

        self.model = keras.Model(inputs=inputs, outputs=outputs)

    def run_training(self):
        self.model.compile(
            optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
            loss=keras.losses.SparseCategoricalCrossentropy(),
            metrics=[keras.metrics.SparseCategoricalAccuracy()])
        
        self.train_history = self.model.fit(
            x=self.train_x, y=self.train_y, epochs=10, batch_size=64, 
            verbose=1, validation_split=0.1)

    def evaluate_pipeline(self):
        self.evaluation = self.model.evaluate(x=self.test_x, y=self.test_y)
        print(self.evaluation)
    
    def __call__(self):
        self.get_data()
        self.build_model()
        self.run_training()
        print(self.keras_model_summary())
        self.evaluate_pipeline()
        super().__call__()

In [ ]:
pipeline_1 = HighLevelPipeline()
pipeline_1()

Epoch 1/10
15/15 [==============================] - 0s 13ms/step - loss: 1.4809 - sparse_categorical_accuracy: 0.5722 - val_loss: 1.0457 - val_sparse_categorical_accuracy: 0.6200
Epoch 2/10
15/15 [==============================] - 0s 3ms/step - loss: 0.6912 - sparse_categorical_accuracy: 0.8189 - val_loss: 0.9689 - val_sparse_categorical_accuracy: 0.6600
Epoch 3/10
15/15 [==============================] - 0s 3ms/step - loss: 0.4773 - sparse_categorical_accuracy: 0.8722 - val_loss: 0.5805 - val_sparse_categorical_accuracy: 0.8400
Epoch 4/10
15/15 [==============================] - 0s 4ms/step - loss: 0.3518 - sparse_categorical_accuracy: 0.9089 - val_loss: 0.5024 - val_sparse_categorical_accuracy: 0.8600
Epoch 5/10
15/15 [==============================] - 0s 4ms/step - loss: 0.2711 - sparse_categorical_accuracy: 0.9278 - val_loss: 0.6103 - val_sparse_categorical_accuracy: 0.8300
Epoch 6/10
15/15 [==============================] - 0s 3ms/step - loss: 0.2337 - sparse_categorical_accuracy:

## Experimenation Log

Model Based
* Using High Level APIs
    * tf.keras functional
* Custom layers
* Custom models
* Custom metrics
* Custom loss
* Custom activation functions
* Custom initializers, regularizers and constants
* Custom training loops
* Custom gradients

Data Loading
 * Tf Example
 * 
Hyperparameter Search


## References
* [Keras for Engineers](https://colab.research.google.com/drive/1lWUGZarlbORaHYUZlF9muCgpPl8pEvve)
* [Keras for Researchers](https://colab.research.google.com/drive/169PfzM0kvtA5UP4k6Sl1yCG9tsE2MLia#scrollTo=4T2NXVLa9Cld)